In [1]:
import numpy as np
import tensorflow as tf

from deeprte.model.tf import input_pipeline

2023-04-07 17:55:33.249050: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-07 17:55:33.299608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/deeprteenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
split = input_pipeline.Split.TRAIN

ds = input_pipeline.load(
    split=split,
    split_ratio=0.8,
    is_training=True,
    batch_sizes=[2, 3],
    collocation_sizes=[120, 30],
)

In [4]:
import jax

dummy_ds = input_pipeline.load(
    split=input_pipeline.Split.TRAIN,
    split_ratio=0.8,
    is_training=True,
    batch_sizes=[jax.local_device_count(), 1],
    collocation_sizes=[1],
    with_info=True,
)

AttributeError: 'tuple' object has no attribute 'with_options'

In [3]:
for i in range(3):
    print(tf.nest.map_structure(lambda x: x.shape, next(iter(ds))))

2023-04-07 17:44:06.260074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_41' with dtype resource
	 [[{{node Placeholder/_41}}]]
2023-04-07 17:44:06.260665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [7]
	 [[{{node Placeholder/_4}}]]


{'boundary': (2, 3, 1920), 'boundary_coords': (2, 3, 1920, 4), 'boundary_scattering_kernel': (2, 3, 1920, 24), 'boundary_weights': (2, 3, 1920), 'phase_coords': (2, 3, 120, 4), 'position_coords': (2, 3, 1600, 2), 'psi_label': (2, 3, 120), 'scattering_kernel': (2, 3, 120, 24), 'self_scattering_kernel': (2, 3, 24, 24), 'sigma': (2, 3, 1600, 2), 'velocity_coords': (2, 3, 24, 2), 'velocity_weights': (2, 3, 24), 'sampled_boundary': (2, 3, 30), 'sampled_boundary_scattering_kernel': (2, 3, 30, 24), 'sampled_boundary_coords': (2, 3, 30, 4), 'sampled_boundary_weights': (2, 3, 30)}
{'boundary': (2, 3, 1920), 'boundary_coords': (2, 3, 1920, 4), 'boundary_scattering_kernel': (2, 3, 1920, 24), 'boundary_weights': (2, 3, 1920), 'phase_coords': (2, 3, 120, 4), 'position_coords': (2, 3, 1600, 2), 'psi_label': (2, 3, 120), 'scattering_kernel': (2, 3, 120, 24), 'self_scattering_kernel': (2, 3, 24, 24), 'sigma': (2, 3, 1600, 2), 'velocity_coords': (2, 3, 24, 2), 'velocity_weights': (2, 3, 24), 'sampled_b